In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import os 
import os.path as osp
import itertools
import astropy.io.fits as fits
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
# from sklearn.model_selection import Kfold
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import f1_score as f1
from sklearn.metrics import recall_score as recall
from sklearn.metrics import precision_score as precision
from sklearn.metrics import classification_report, make_scorer

In [ ]:
# loading the data from the machine
file = osp.join("COSMOSXMATCH+classes_040422_withphotometry.fits")
fp = fits.open(file, memmap=True)
head = fp[1].header

data = fp[1].data
fp.close

# number of elements in the data
N_all = len(data)

In [ ]:
# DATA SAMPLING AND PREPROCESSING
def data_processor( data, x_features, y_features, binary_classification = True ):
    """ This function prepares the samples a subset of features for machine learning from a high dimensional data.
        classification : type of classification, binary == SFG or AGNs, Other wise classification will be for AGNs
        or SFG or noclass
        data = high dimensional dataset (MIGHTEE data)
        X-features = columns of interest contain data
        Y-features = the output features 
        
    """
    if binary_classification == True:
        # extracting the x-features
        x_sets = []
        for i in x_features:
            x = data[i]
            x_sets.append(np.array(x))
    
        X = np.vstack((x_sets)).T
    
        #extracting the y-feature
        y_sets = []
        for l in y_features:
            y = data[l]
            y_sets.append(y)
        
        y = np.vstack((y_sets)).T
    
        # converting the features into the data frame
        y_data = pd.DataFrame(y, columns = y_features)
        X_data = pd.DataFrame(X, columns = x_features)

        # joinin the two data sets into one dataframe
        mightee_data = pd.concat([X_data, y_data], axis=1, sort=False)
    
        # Sampling the sources that are classified as midIRAGB = AGN and the sources that are classified as notmidIRAGN = SFG
        AGN = mightee_data[mightee_data[y_features[0]] == True]
        SFG = mightee_data[mightee_data[y_features[1]] == True]
        probSFG = mightee_data[mightee_data[y_features[2]] == True]
    
        print('shape of the AGN: ', AGN.shape)
        print('shape of the SFG: ', SFG.shape)
        print('shape of the probSFG: ', probSFG.shape)
        print('total sample: ', len(AGN) + len(SFG)+len(probSFG))
    
        # We now drop the not column
        mightee_agn = AGN.drop([y_features[1], y_features[2]], axis = 1)
        mightee_sfg = SFG.drop([y_features[1], y_features[2]], axis = 1)
        mightee_probsfg = probSFG.drop([y_features[1], y_features[2]], axis = 1)

        # We now replace True with the true label AGN or SFG for the corresponding source
        mightee_agn1 = mightee_agn.replace(True, 'AGN', regex=True)
        mightee_sfg1 = mightee_sfg.replace(False, 'SFG', regex=True)
        mightee_probsfg1 = mightee_probsfg.replace(False, 'SFG', regex=True)
    
        # combining this data into one
        complete_mightee = pd.concat([mightee_agn1, mightee_sfg1, mightee_probsfg1], sort=False)
        complete_mightee1 = complete_mightee.replace(-np.inf, np.nan, regex=True) 
        print("DONE PROCESSING")
    
        return complete_mightee1





    else:
        x_sets = []
        for i in x_features:
            x = data[i]
            x_sets.append(np.array(x))

        X = np.vstack((x_sets)).T

        #extracting the y-feature
        y_sets = []
        for l in y_features:
            y = data[l]
            y_sets.append(y)

        y = np.vstack((y_sets)).T

        # converting the features into the data frame
        y_data = pd.DataFrame(y, columns = y_features)
        X_data = pd.DataFrame(X, columns = x_features)

        # joinin the two data sets into one dataframe
        mightee_data = pd.concat([X_data, y_data], axis=1, sort=False)

        # Sampling the sources that are classified as midIRAGB = AGN and the sources that are classified as notmidIRAGN = SFG
        AGN = mightee_data[mightee_data[y_features[0]] == True]
        SFG = mightee_data[mightee_data[y_features[1]] == True]
        probSFG = mightee_data[mightee_data[y_features[2]] == True]
        noclass = mightee_data[(mightee_data[y_features[0]] == False) & (mightee_data[y_features[1]] == False) & (mightee_data[y_features[2]] == False)]

        print('shape of the AGN: ', AGN.shape)
        print('shape of the SFG: ', SFG.shape)
        print('shape of the probSFG: ', probSFG.shape)
        print('shape of unclassified: ',noclass.shape)
        print('total sample: ', len(AGN) + len(SFG) + len(probSFG) + len(noclass))

        # We now drop the not column
        mightee_agn = AGN.drop([y_features[1], y_features[2]], axis = 1)
        mightee_sfg = SFG.drop([y_features[1], y_features[2]], axis = 1)
        mightee_probsfg = probSFG.drop([y_features[1], y_features[2]], axis = 1)
        mightee_noclass = noclass.drop([y_features[1], y_features[2]], axis = 1)

        # We now replace True with the true label AGN or SFG for the corresponding source
        mightee_agn1 = mightee_agn.replace(True, 'AGN', regex=True)
        mightee_sfg1 = mightee_sfg.replace(False, 'SFG', regex=True)
        mightee_probsfg1 = mightee_probsfg.replace(False, 'SFG', regex=True)
        mightee_noclass1 = mightee_noclass.replace(False, 'noclass', regex=True)

        # combining this data into one
        complete_mightee = pd.concat([mightee_agn1, mightee_sfg1, mightee_probsfg1, mightee_noclass1], sort=False)
        complete_mightee1 = complete_mightee.replace(-np.inf, np.nan, regex=True) 
        print("DONE PROCESSING")
    
        return complete_mightee1

In [ ]:
# Important data columns for machine learning

x_fea = ['COS_best_z_v5', 'SPLASH_1_FLUX', 'SPLASH_2_FLUX', 'SPLASH_3_FLUX', 'SPLASH_4_FLUX',
            'L14','LIR_WHz','MASS_lephare', 'class_star','qir','flux_HSC-G','flux_HSC-R', 'flux_HSC-I','flux_HSC-Z',
            'flux_J', 'flux_H', 'flux_Ks', 'flux_Y']
y_fea = ['AGN', 'SFG', 'probSFG']


trad_flux = data_processor(data, x_fea, y_fea)

In [ ]:
# We rename AGN column to labels

trad_flux.rename(columns = {'AGN':'class_labels', 'MASS_lephare':'Mstar'}, inplace = True)

trad_flux_clean1 = trad_flux.dropna()
trad_flux_clean = trad_flux_clean1.drop(['class_star', 'qir'], axis = 1)

In [ ]:
x_cols = ['COS_best_z_v5', 'SPLASH_1_FLUX', 'SPLASH_2_FLUX', 'SPLASH_3_FLUX', 'SPLASH_4_FLUX',
            'L14','LIR_WHz','Mstar','flux_HSC-G','flux_HSC-R', 'flux_HSC-I','flux_HSC-Z',
            'flux_J', 'flux_H', 'flux_Ks', 'flux_Y']

x_cols_no_mstar = ['COS_best_z_v5', 'SPLASH_1_FLUX', 'SPLASH_2_FLUX', 'SPLASH_3_FLUX', 'SPLASH_4_FLUX',
            'L14','LIR_WHz','flux_HSC-G','flux_HSC-R', 'flux_HSC-I','flux_HSC-Z',
            'flux_J', 'flux_H', 'flux_Ks', 'flux_Y']
x_trad = trad_flux_clean[x_cols]

y_trad = trad_flux_clean['class_labels']

# encoding target class
y, clas = pd.factorize(y_trad)
y_target = pd.DataFrame(y, columns = ['labels'])

X_train, X_test, y_train, y_test = train_test_split(x_trad, y_target, stratify = y, test_size=0.25, random_state=42)

In [ ]:
# The trad Fluxes

S8_S45 = np.log10( trad_flux_clean1['SPLASH_4_FLUX'] / trad_flux_clean1['SPLASH_2_FLUX'])
S58_S36 = np.log10( trad_flux_clean1['SPLASH_3_FLUX'] / trad_flux_clean1['SPLASH_1_FLUX'])
S45_S36 = np.log10( trad_flux_clean1['SPLASH_2_FLUX'] / trad_flux_clean1['SPLASH_1_FLUX'])


trad_flux_clean1['log(S8/S45)'] = S8_S45
trad_flux_clean1['log(S58/S36)'] = S58_S36
# trad_flux_clean1['log(S58/S36)'] = S8_S45
# trad_flux_clean1['log(S8/S45)'] = S58_S36
trad_flux_clean1['log(S45/S36)'] = S45_S36

In [ ]:
# use color as features
# magnitudes between two different filter bands.
# features: [u-g, g-r, r-i, i-z]
g_hsc = trad_flux_clean1['flux_HSC-G']
r_hsc = trad_flux_clean1['flux_HSC-R']
i_hsc = trad_flux_clean1['flux_HSC-I']
z_hsc = trad_flux_clean1['flux_HSC-Z']   
    
# feaures
g_r, r_i, i_z, g_i, g_z, r_z = np.array(np.log10(g_hsc / r_hsc)), np.array(np.log10(r_hsc /i_hsc)), np.array(np.log10(i_hsc /z_hsc)), np.array(np.log10(g_hsc / i_hsc)), np.array(np.log10(g_hsc / z_hsc)), np.array(np.log10(r_hsc / z_hsc))

trad_flux_clean1['log(g/r)'], trad_flux_clean1['log(r/i)'], trad_flux_clean1['log(i/z)'], trad_flux_clean1['log(g/i)'], trad_flux_clean1['log(g/z)'], trad_flux_clean1['log(r/z)']  =  g_r, r_i, i_z, g_i, g_z, r_z

In [ ]:
# The NIR colours

y_hsc = trad_flux_clean1['flux_Y']
h_hsc = trad_flux_clean1['flux_H']
k_hsc = trad_flux_clean1['flux_Ks']
j_hsc = trad_flux_clean1['flux_J']
    
    
# feaures
y_j, j_h, h_k, y_h, y_k, j_k = np.array(np.log10(y_hsc / j_hsc)), np.array(np.log10(j_hsc /h_hsc)), np.array(np.log10(h_hsc / k_hsc)), np.array(np.log10(y_hsc / h_hsc)), np.array(np.log10(y_hsc / k_hsc)), np.array(np.log10(j_hsc / k_hsc))
# hsc_columns = ['y/j', 'j/h', 'h/k']
trad_flux_clean1['log(Y/J)'], trad_flux_clean1['log(J/H)'], trad_flux_clean1['$log(H/K_{s})$'], trad_flux_clean1['log(Y/H)'], trad_flux_clean1['$log(Y/K_{s})$'],trad_flux_clean1['$log(J/K_{s})$'] =  y_j, j_h, h_k,y_h, y_k, j_k

In [ ]:
colors_data = trad_flux_clean1.drop(x_cols_no_mstar, axis = 1)
colors_data1 = colors_data.replace(-np.inf, np.nan, regex=True)
colors = colors_data1.dropna()

In [ ]:
colors

In [ ]:
# # Generate sample data (replace this with your actual dataset)
# X = colors.drop('class_labels', axis=1)
# y = colors['class_labels']
# # encoding target class
# y1, clas1 = pd.factorize(y)
# target = pd.Series(y1)  # Changed to Series to avoid warning

# feature_names = X.columns.tolist()  # Convert to list

# def iterative_feature_importance(X, y, feature_names):
#     """
#     Calculate feature importance iteratively by removing the most important feature at each step
    
#     Args:
#     X: numpy array or pandas DataFrame of features
#     y: numpy array or pandas Series of target variable
#     feature_names: list of feature names
    
#     Returns:
#     Dictionary containing importance values at each iteration
#     """
#     X = pd.DataFrame(X, columns=feature_names)
#     remaining_features = feature_names.copy()  # This is now a list
#     importance_results = {}
    
#     iteration = 0
#     while len(remaining_features) > 0:
#         # Train Random Forest
#         rf = RandomForestClassifier(n_estimators=100, random_state=42)
#         rf.fit(X[remaining_features], y)
        
#         # Get feature importances
#         importances = rf.feature_importances_
#         importance_dict = dict(zip(remaining_features, importances))
        
#         # Store results
#         importance_results[f'Iteration_{iteration}'] = {
#             'features': remaining_features.copy(),
#             'importances': importance_dict
#         }
        
#         # Find and remove the most important feature (if more than one feature left)
#         if len(remaining_features) > 1:
#             most_important = max(importance_dict, key=importance_dict.get)
#             remaining_features.remove(most_important)
#             print(f"Iteration {iteration}: Removed {most_important}")
#         else:
#             print(f"Final iteration: Only {remaining_features[0]} remains")
#             break
            
#         iteration += 1
    
#     return importance_results

# # Run the analysis
# results = iterative_feature_importance(X, target, feature_names)

# # Plot the results
# plt.figure(figsize=(12, 8))

# for i, (key, value) in enumerate(results.items()):
#     features = value['features']
#     importances = value['importances']
    
#     # Sort features by importance
#     sorted_features = sorted(importances.items(), key=lambda x: x[1], reverse=True)
#     features_sorted = [f[0] for f in sorted_features]
#     importance_values = [f[1] for f in sorted_features]
    
#     plt.subplot(len(results), 1, i+1)
#     plt.barh(features_sorted, importance_values)
#     plt.title(f'{key} - {len(features)} features remaining')
#     plt.xlabel('Importance Score')
#     plt.xlim(0, 1)

# plt.tight_layout()
# plt.show()

# # Print results in a format similar to your screenshot
# print("\n# Importance")
# for i, (key, value) in enumerate(results.items()):
#     features = value['features']
#     importances = value['importances']
    
#     if i == 0:
#         print(f"- **Input**: {importances}")
#     else:
#         print(f"- **Output after removing top feature**: {importances}")
    
#     if i < len(results) - 1:
#         print("---")

In [ ]:
# Generate sample data
X = colors.drop('class_labels', axis=1)
y = colors['class_labels']
# encoding target class
y1, clas1 = pd.factorize(y)
target = pd.Series(y1)  # Using Series to avoid warning

feature_names = X.columns.tolist()  # Convert to list

def iterative_feature_importance(X, y, feature_names):
    """
    Calculate feature importance iteratively by removing the most important feature at each step
    """
    X = pd.DataFrame(X, columns=feature_names)
    remaining_features = feature_names.copy()
    importance_results = {}
    
    iteration = 0
    while len(remaining_features) > 0:
        # Train Random Forest
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X[remaining_features], y)
        
        # Get feature importances
        importances = rf.feature_importances_
        importance_dict = dict(zip(remaining_features, importances))
        
        # Store results
        importance_results[f'Iteration {iteration}'] = {
            'features': remaining_features.copy(),
            'importances': importance_dict,
            'n_features': len(remaining_features)
        }
        
        # Find and remove the most important feature
        if len(remaining_features) > 1:
            most_important = max(importance_dict, key=importance_dict.get)
            remaining_features.remove(most_important)
            print(f"Iteration {iteration}: Removed {most_important}")
        else:
            print(f"Final iteration: Only {remaining_features[0]} remains")
            break
            
        iteration += 1
    
    return importance_results

# Run the analysis
results = iterative_feature_importance(X, target, feature_names)

# Plot the results in a grid with 3 columns
n_iterations = len(results)
n_cols = 3
n_rows = (n_iterations + n_cols - 1) // n_cols  # Calculate needed rows

plt.figure(figsize=(18, 5 * n_rows))  # Adjust figure size based on rows

for i, (key, value) in enumerate(results.items()):
    features = value['features']
    importances = value['importances']
    n_features = value['n_features']
    
    # Sort features by importance
    sorted_features = sorted(importances.items(), key=lambda x: x[1], reverse=True)
    features_sorted = [f[0] for f in sorted_features]
    importance_values = [f[1] for f in sorted_features]
    
    # Create subplot (n_rows, n_cols, i+1)
    plt.subplot(n_rows, n_cols, i+1)
    bars = plt.barh(features_sorted, importance_values)
    plt.title(f'{key}\n({n_features} features remaining)')
    plt.xlabel('Importance Score')
    plt.xlim(0, max(importance_values) * 1.1)  # Add 10% padding
    
    # Add value labels on bars
    for bar in bars:
        width = bar.get_width()
        plt.text(width, bar.get_y() + bar.get_height()/2,
                f'{width:.3f}',
                ha='left', va='center')

plt.tight_layout()
plt.savefig('RF-importance.pdf')
plt.show()

# Print results summary
print("\n# Feature Importance Analysis")
for i, (key, value) in enumerate(results.items()):
    features = value['features']
    importances = value['importances']
    
    if i == 0:
        print(f"- **Initial feature importances**:")
    else:
        print(f"- **After iteration {i} (removed {most_important_removed})**:")
        most_important_removed = list(set(prev_features) - set(features))[0]
    
    for feat, imp in sorted(importances.items(), key=lambda x: x[1], reverse=True):
        print(f"  - {feat}: {imp:.4f}")
    
    if i < len(results) - 1:
        print("---")
    prev_features = features.copy()